## 11. Transformix: Spatial Jacobian and Deformation Field calculations

With the transformix algorithm the spatial jacobian, the determinant of the spatial jacobian and the deformation field of the transformation can be calculated.
Especially the determinant of the spatial Jacobian, which identifies the amount of local
compression or expansion and can be quite useful, for example in lung ventilation studies.

### Elastix

In [1]:
from itk import itkElastixRegistrationMethodPython
from itk import itkTransformixFilterPython
import itk

In [2]:
# Import Images
fixed_image = itk.imread('data/CT_2D_head_fixed.mha', itk.F)
moving_image = itk.imread('data/CT_2D_head_moving.mha', itk.F)

# Import Default Parameter Map
parameter_object = itk.ParameterObject.New()
parameter_map_rigid = parameter_object.GetDefaultParameterMap('rigid')
parameter_object.AddParameterMap(parameter_map_rigid)

Registration with the registration function...

In [3]:
# Call registration function
result_image, result_transform_parameters = itk.elastix_registration_method(
    fixed_image, moving_image,
    parameter_object=parameter_object)

### Transformix

In [9]:
# Import Image to transform;
# for this example the exact same moving image is used, this however is normally not 
# very usefull since the elastix algorithm already transformed this image.
moving_image_transformix = itk.imread('data/CT_2D_head_moving.mha', itk.F)

Transformation can either be done in one line with the transformix function...

In [10]:
transformed_image = itk.transformix_filter(
    moving_image=moving_image,
    transform_parameter_object=result_transform_parameters,
    compute_deformation_field=True,
    compute_spatial_jacobian=True,
    compute_determinant_of_spatial_jacobian=True,
    output_directory='exampleoutput/')

.. or by initiating an transformix image filter object.

In [11]:
# Load Transformix Object
transformix_object = itk.TransformixFilter.New()
transformix_object.SetMovingImage(moving_image_transformix)
transformix_object.SetTransformParameterObject(result_transform_parameters)

# Set advanced options
transformix_object.SetComputeSpatialJacobian(True)
transformix_object.SetComputeDeterminantOfSpatialJacobian(True)
transformix_object.SetComputeDeformationField(True)

# Set output directory for spatial jacobian and its determinant,
# default directory is current directory.
transformix_object.SetOutputDirectory('exampleoutput/')

# Update object (required)
transformix_object.UpdateLargestPossibleRegion()

# Results of Transformation
result_image_transformix = transformix_object.GetOutput()
deformation_field = transformix_object.GetOutputDeformationField()

The deformation field can currently only be computed with the object oriented method, this will change is future updates to ITKElastix